In [4]:
import json
import requests
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import re


In [5]:
spotify_client = os.environ.get('SPOTIFY_CLIENT')
spotify_secret = os.environ.get('SPOTIFY_SECRET')

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_client,  
                                               client_secret=spotify_secret,
                                               redirect_uri='http://localhost:8888/callback/',
                                               scope='user-read-currently-playing'))


In [17]:
current_song = sp.currently_playing(market='FR', additional_types='track')

print(current_song)

{'timestamp': 1605290516458, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/7tPSBvTBEh7qJ4X192u9uP'}, 'href': 'https://api.spotify.com/v1/playlists/7tPSBvTBEh7qJ4X192u9uP', 'type': 'playlist', 'uri': 'spotify:user:dxzag7le43qidxq3x1j6ggows:playlist:7tPSBvTBEh7qJ4X192u9uP'}, 'progress_ms': 9692, 'item': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7wSr4otUV4ytUvHE4w3rBe'}, 'href': 'https://api.spotify.com/v1/artists/7wSr4otUV4ytUvHE4w3rBe', 'id': '7wSr4otUV4ytUvHE4w3rBe', 'name': 'Hovey Benjamin', 'type': 'artist', 'uri': 'spotify:artist:7wSr4otUV4ytUvHE4w3rBe'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/2IxLA2DWXN3WnSgoBpZiDX'}, 'href': 'https://api.spotify.com/v1/albums/2IxLA2DWXN3WnSgoBpZiDX', 'id': '2IxLA2DWXN3WnSgoBpZiDX', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b2730cc011894c226a095cd2f47b', 'width': 640}, {'height': 300, 'url': 'https://

In [18]:
flat_df = pd.io.json.json_normalize(current_song)
flat_df

,timestamp,progress_ms,currently_playing_type,is_playing,context.external_urls.spotify,context.href,context.type,context.uri,item.album.album_type,item.album.artists,...,item.id,item.is_local,item.is_playable,item.name,item.popularity,item.preview_url,item.track_number,item.type,item.uri,actions.disallows.resuming
0,1605290516458,9692,track,True,https://open.spotify.com/playlist/7tPSBvTBEh7q...,https://api.spotify.com/v1/playlists/7tPSBvTBE...,playlist,spotify:user:dxzag7le43qidxq3x1j6ggows:playlis...,single,[{'external_urls': {'spotify': 'https://open.s...,...,0m9QNpZ4XVqkwjhPDXAtmP,False,True,Never Too Much,32,https://p.scdn.co/mp3-preview/7b998c9060b3bae6...,1,track,spotify:track:0m9QNpZ4XVqkwjhPDXAtmP,True


In [19]:
playlist = re.split('/', flat_df['context.href'][0])[5]
track_name = flat_df['item.name'][0]
track_artist = flat_df['item.artists'][0][0]['name']

#flat_df.columns
#track_name
playlist
#track_artist
#flat_df['context.href'][0]


'7tPSBvTBEh7qJ4X192u9uP'

In [37]:
import lyricsgenius

genius = lyricsgenius.Genius(os.environ.get('GENIUS_TOKEN'))
song = genius.search_song(track_name, track_artist, get_full_info=False)

Searching for "Are You Bored Yet? (feat. Clairo)" by Wallows...
Done.


In [38]:
print(song.lyrics)

[Verse 1: Dylan Minnette with 
Clairo
]
What's wrong?
You've been asking but I don't have an answer
How come?
I'm still thinking, let's pretend to fall asleep now
When we get old, will we regret this?
Too young to think about all that shit
And stalling only goes so far when you've got a head start

[Chorus: Dylan Minnette with 
Clairo
]
'Cause we could stay at home and 
watch the sunset
But I can't help from asking, 
"Are you bored yet?"
And if you're feeling lonely 
you should tell me
Before this ends up as 
another memory
Will you tell the truth so I don't have to lie?
Will you tell the truth so I don't have to lie?

[Verse 2: Clairo]
Feels like I've known you my whole life
I can see right through your lies
I don't know where we're going
But I'd like to be by your side
If you could tell me how you're feeling
Maybe we'd get through this undefeated
Holding on for so long

[Chorus: Dylan Minnette with 
Clairo
]
'Cause we could stay at home and 
watch the sunset
But I can't help from ask

In [3]:
from twilio.rest import Client 

account_sid = os.environ.get('TWILIO_ACCOUNT')
auth_token = os.environ.get('TWILIO_TOKEN') 
client = Client(account_sid, auth_token) 

message = client.messages.create(
                    from_= 'whatsapp:+14155238886', #TWILIO's WhatsApp number 
                    body= 'Hello!',      
                    to= 'whatsapp:' + os.environ.get('SARAH_NUMBER'))